# YNAB Analysis

Currently YNAB does not provide any meaningful analysis for understanding where your current spending is over/under against your bugdet. Furthermore, the budget view resets every month, so overages disappear and can get lost, leading to an increase in incramental credit card debt.

This notebook is the start of ingesting the register data out of YNAB and doing a two week lookback (one pay period).

Questions this aims to answer:
* Is the spend habitual or one-off?
* What payees seem to be causing the problems?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import config # contains sensative data (see note below)
from scipy.stats import linregress

In [ ]:
# Import register data
df = pd.read_csv('2023-06-14 17-34 - Register.csv')

# Convert 'Outflow' and 'Inflow' columns to numeric
df['Outflow'] = df['Outflow'].str.replace('$', '').str.replace(',', '').astype(float)
df['Inflow'] = df['Inflow'].str.replace('$', '').str.replace(',', '').astype(float)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Examine data import
df.head()

In [ ]:
# List unique accounts to determine if any need to be filtered out
df['Account'].unique()

## Top 10 payee analysis

**Note:**
Sensitive data is contained within a `config.py` file. Template of `config.py` provided in the repo.

In [ ]:
# Define the payees to be filtered out
excluded_payees = config.excluded_payees

# Replace missing values in 'Payee' column with an empty string
df['Payee'] = df['Payee'].fillna('')

# Extract the variables from the config module
start_date = config.start_date
end_date = config.end_date
excluded_payees = config.excluded_payees
excluded_prefixes = config.excluded_prefixes
excluded_categories = config.excluded_categories
excluded_accounts = config.excluded_accounts

# Filter the dataset by the date range and specific payees
filtered_df = (df[(df['Date'] >= start_date) &
                  (df['Date'] <= end_date) &
                  (~df['Payee'].isin(excluded_payees)) &
                  (~df['Payee'].str.startswith(excluded_prefixes)) &
                  (df['Category'] != excluded_categories) &
                  (df['Account'] != excluded_accounts)])

# Group the filtered data by payee and category and aggregate the outflow
summary = filtered_df.groupby(['Payee'])['Outflow'].sum().reset_index()

# Sort the summary data by payee in descending order
summary = summary.sort_values('Outflow', ascending=False)

# Filter the top ten payees
top_ten_payees = summary.head(10)

# Plot the summarized data for the top ten payees
plt.figure(figsize=(10, 6))
bars = plt.bar(top_ten_payees['Payee'], top_ten_payees['Outflow'])
plt.xlabel('Payee')
plt.ylabel('Outflow')
plt.title('Top Ten Payees - Summarized Outflow (Last Two Weeks)')
plt.xticks(rotation=90)

# Add the amount on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"${height:.2f}", ha='center', va='bottom')

plt.show()

## Top 10 category analysis

In [ ]:
# Group the filtered data by category and aggregate the outflow
summary = filtered_df.groupby('Category')['Outflow'].sum().reset_index()

# Sort the summary data by outflow in descending order
summary = summary.sort_values('Outflow', ascending=False)

# Filter the top ten categories
top_ten_categories = summary.head(10)

# Plot the summarized data for the top ten categories
plt.figure(figsize=(10, 6))
bars = plt.bar(top_ten_categories['Category'], top_ten_categories['Outflow'])
plt.xlabel('Category')
plt.ylabel('Outflow')
plt.title('Top Ten Categories - Summarized Outflow (Last Two Weeks)')
plt.xticks(rotation=90)

# Add the amount on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"${height:.2f}", ha='center', va='bottom')

plt.show()

## Regression on Day of Week vs. Outflow

(No real meaningful output)

In [ ]:
# Extract the week of the year and day of the week
df['WeekOfYear'] = df['Date'].dt.week
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Perform regression analysis for WeekOfYear vs Outflow
regression_week = pd.DataFrame({'WeekOfYear': df['WeekOfYear'], 'Outflow': df['Outflow']})
results_week = regression_week.groupby('WeekOfYear').mean().reset_index()

# Perform regression analysis for DayOfWeek vs Outflow
regression_day = pd.DataFrame({'DayOfWeek': df['DayOfWeek'], 'Outflow': df['Outflow']})
results_day = regression_day.groupby('DayOfWeek').mean().reset_index()

# Fit a linear regression model for WeekOfYear vs Outflow
slope_week, intercept_week, r_value_week, p_value_week, std_err_week = linregress(results_week['WeekOfYear'], results_week['Outflow'])
regression_line_week = intercept_week + slope_week * results_week['WeekOfYear']

# Fit a linear regression model for DayOfWeek vs Outflow
slope_day, intercept_day, r_value_day, p_value_day, std_err_day = linregress(results_day['DayOfWeek'], results_day['Outflow'])
regression_line_day = intercept_day + slope_day * results_day['DayOfWeek']

# Plot the regression results for WeekOfYear vs Outflow
plt.figure(figsize=(10, 6))
plt.scatter(results_week['WeekOfYear'], results_week['Outflow'])
plt.plot(results_week['WeekOfYear'], regression_line_week, color='red', label='Regression Line')
plt.xlabel('Week of the Year')
plt.ylabel('Outflow')
plt.title('Regression Analysis: Week of the Year vs Outflow')
plt.legend()
plt.show()

# Plot the regression results for DayOfWeek vs Outflow
plt.figure(figsize=(10, 6))
plt.scatter(results_day['DayOfWeek'], results_day['Outflow'])
plt.plot(results_day['DayOfWeek'], regression_line_day, color='red', label='Regression Line')
plt.xlabel('Day of the Week')
plt.ylabel('Outflow')
plt.title('Regression Analysis: Day of the Week vs Outflow')
plt.xticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.legend()
plt.show()

## Friday transaction analysis

In theory, if I'm paid on a Friday, I might tend to spend more when I'm cash flush. Are there more impulsive transactions occuring at this time?

In [ ]:
# Filter the dataset for transactions that occurred on Fridays
friday_transactions = df[df['Date'].dt.dayofweek == 4]  # 4 represents Friday

# Aggregate the outflow by category for Friday transactions
category_outflow = friday_transactions.groupby('Category')['Outflow'].sum().reset_index()

# Sort the categories by aggregated outflow in descending order
category_outflow = category_outflow.sort_values('Outflow', ascending=False)

# Plot the top areas of spending on Fridays
plt.figure(figsize=(10, 6))
bars = plt.bar(category_outflow['Category'], category_outflow['Outflow'])
plt.xlabel('Category')
plt.ylabel('Outflow')
plt.title('Top Areas of Spending on Fridays')
plt.xticks(rotation=90)

# Add the amount on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"${height:.2f}", ha='center', va='bottom')

plt.show()
